In [1]:
# Import necessary libraries

from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

def init_browser():
    
    executable_path ={'executable_path': 'chromedriver'}
    return Browser('chrome', **executable_path, headless=False)

def scrape():
    browser = init_browser()
    # listings = {}
    # NASA Mars News
    url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    news_title = soup.find('div', class_="content_title").text
    news_text = soup.find('div', class_="article_teaser_body").text
    
    # 2. JPL Mars Space Images - Featured Image
    url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    imgbody = soup.find("a", class_="fancybox").get('data-fancybox-href').strip()
    imgurl = 'https://www.jpl.nasa.gov'+imgbody
    
    # 3. Mars Weather
    url = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    d0xha r-ad9z0x r-bcqeeo r-qvutc0"
    tweet = soup.find(class_="js-tweet-text").text
    weather = tweet.split('pic')[0] 
    
    # 4. Mars Facts
    url = 'https://space-facts.com/mars/'
    tables = pd.read_html(url)
    df = tables[0]
    fact_list=df.values.tolist()
    
    # 5. Mars Hem
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    url_list = []
    itemlinks = soup.find_all(class_="description")

    for itemlink in itemlinks:
        item = itemlink.find('a', class_="itemLink product-item").get('href')
        url_list.append(item)
    item_url_list = ['https://astrogeology.usgs.gov/' + item_url for item_url in url_list]
    
    title_list = []
    image_urls_list = []
    for x in range (0,4):

        item_url = item_url_list[x]
        browser.visit(item_url)
        html = browser.html
        soup = bs(html, 'html.parser')
    #     print(item_url)
        image_urls = soup.find("li").find('a')['href']
        title = soup.find('h2', class_="title").text
        title_list.append(title)
        image_urls_list.append(image_urls)

    listings = [{'news':news_title}, 
           {'text':news_text}, 
           {'url':imgurl}, 
           {'weather':weather},
           {'description':fact_list[0][0], 'value':fact_list[0][1]},
           {'description':fact_list[1][0], 'value':fact_list[1][1]},
           {'description':fact_list[2][0], 'value':fact_list[2][1]},
           {'description':fact_list[3][0], 'value':fact_list[3][1]},
           {'description':fact_list[4][0], 'value':fact_list[4][1]},
           {'description':fact_list[5][0], 'value':fact_list[5][1]},
           {'description':fact_list[6][0], 'value':fact_list[6][1]},
           {'description':fact_list[7][0], 'value':fact_list[7][1]},
           {'description':fact_list[8][0], 'value':fact_list[8][1]},
           {"title": title_list[0], "img_url": image_urls_list[0]},                       
           {"title": title_list[1], "img_url": image_urls_list[1]},
           {"title": title_list[2], "img_url": image_urls_list[2]},
           {"title": title_list[3], "img_url": image_urls_list[3]}
           ]
    browser.quit()
    return listings



In [2]:
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import scrape_mars_copy
import pymongo

app = Flask(__name__)

# use flask_pymongo to set up mongo connection

# app.config["MONGO_URI"] = "mongodb://localhost:27017/mars_app"
# mongo = PyMongo(app)

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.mars_app
db.listings.drop()

@app.route("/")
def index():
    # listings_test = list(db.listings.find())
    # print(listings_test)
    listings_test =list(db.listings.find({"news": {"$exists": True}}))
    listings_test1 =list(db.listings.find({"text": {"$exists": True}}))
    listings_test2 =list(db.listings.find({"url": {"$exists": True}}))
    listings_test3 =list(db.listings.find({"weather": {"$exists": True}}))
    listings_test4 =list(db.listings.find({"description": {"$exists": True}}))
    listings_test5 =list(db.listings.find({"title": {"$exists": True}}))
    # print (listings_test)

    return render_template("index.html", listings=listings_test, listings1=listings_test1,listings2=listings_test2,
                            listings3=listings_test3,listings4=listings_test4,listings5=listings_test5)
    
@app.route("/scrape")
def scraper():
    db.listings.drop()
    listings = db.listings
    listings_data = scrape_mars_copy.scrape()
    listings.insert_many(listings_data)
    # listings.update({}, listings_data, upsert=True)
    return redirect("/", code=302)

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
